#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [2]:
import pymysql
from sqlalchemy import create_engine
import pymysql.cursors

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [3]:

#cursors is the pointer and gives you the result
connection = pymysql.connect(host='relational.fit.cvut.cz',
                            user='guest',
                            password='relational',
                            db='stats',
                            charset='utf8mb4',
                            cursorclass=pymysql.cursors.DictCursor)

#### 4. Import the users table 

In [4]:
data=pd.read_sql_query('SELECT * FROM stats.users LIMIT 100', connection)


#### 5. Rename Id column to userId

In [5]:
data.rename(columns = {'Id':'userId'}, inplace = True)


#### 6. Import the posts table. 

In [6]:
data5=pd.read_sql_query('SELECT * FROM stats.posts LIMIT 100', connection)


#### 7. Rename Id column to postId and OwnerUserId to userId

In [7]:
data5 = data5.rename(columns={'Id': 'postId', 'OwnerUserId': 'userId'})

#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [14]:
ucolnames = ['userId','Reputation','Views','UpVotes','DownVotes']
pcolnames = ['postId','Score','userId','ViewCount','CommentCount']
newuser= pd.DataFrame(data,columns=ucolnames)
newposts = pd.DataFrame(data5,columns=pcolnames)

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [18]:
mergeframe = newuser.merge(newposts, left_on='userId', right_on='userId', how ='left')
mergeframe.head(10)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,NaN,NaN,NaN,NaN
1,2,101,25,3,0,NaN,NaN,NaN,NaN
2,3,101,22,19,0,NaN,NaN,NaN,NaN
3,4,101,11,0,0,NaN,NaN,NaN,NaN
4,5,6792,1145,662,5,6.0,152.0,29229.0,5.0
5,5,6792,1145,662,5,12.0,20.0,NaN,1.0
6,5,6792,1145,662,5,32.0,12.0,NaN,0.0
7,5,6792,1145,662,5,49.0,6.0,NaN,0.0
8,5,6792,1145,662,5,64.0,6.0,NaN,0.0
9,5,6792,1145,662,5,76.0,22.0,NaN,3.0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [22]:
null_cols = mergeframe.isnull().sum()
null_cols[null_cols > 0]

count_nan = len(mergeframe) - mergeframe.count()
print(count_nan)
mergeframe.shape

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId          40285
Score           40285
ViewCount       40345
CommentCount    40285
dtype: int64


(40381, 9)

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [23]:
#Due to the amount of missing values (40345-40285) compared to the size of the database (40381) , I'll drop them. 
drop_cols = list(null_cols[null_cols > 10000].index)
datamerge = mergeframe.drop(drop_cols, axis=1)

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [ ]:
#PostID, Score, ViewCount, CommentCount

#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.